In [2]:
# 读取数据
import pandas as pd

# 数据集文件路径
file_path = '../../../../dataset/compare/covtype_with_key.subset.data'

# 使用列名列表读取数据
data = pd.read_csv(file_path)
# 获取Cover_Type的值域（唯一值）
unique_values = data['Cover_Type'].unique()

# 获取每个值的计数
value_counts = data['Cover_Type'].value_counts()

# 打印结果
print("Cover_Type的值域: ", unique_values)
print("\n每个值的大小:")
print(value_counts)


Cover_Type的值域:  [5 2 1 7 3 6 4]

每个值的大小:
Cover_Type
5    2000
2    2000
1    2000
7    2000
3    2000
6    2000
4    2000
Name: count, dtype: int64


## 嵌入水印

In [3]:
# 定义用户私钥 Ks
import secrets

# 生成 16 bits 的随机密钥
Ks = secrets.token_hex(2) # 用户私钥
k = secrets.token_hex(2) # attribute 对应的私钥

print("Generated 16 bits Ks: ", Ks)
print("Generated 16 bits k: ", k)

# 保存 Ks 到文件
with open('semantic_Ks.txt', 'w') as file:
    file.write(Ks)

# 保存 k 到文件
with open('semantic_k.txt', 'w') as file:
    file.write(k)

# 生成嵌入水印信息

import random

# 生成一个长度为 400 的字符串，它由随机的 0 和 1 组成
watermark = ''.join(random.choice('01') for _ in range(400))
# 打印二进制字符串
print("要嵌入的水印信息是",watermark)

# 统计二进制字符串中 1 的个数
count_1 = watermark.count('1')

# 统计二进制字符串中 0 的个数
count_0 = watermark.count('0')

# 打印结果
print("Count of 1: ", count_1)
print("Count of 0: ", count_0)


# 保存水印信息到文件
with open("semantic_mark.txt", "w") as file:
    file.write(watermark)

print("水印信息已保存到 semantic_mark.txt")

Generated 16 bits Ks:  b80d
Generated 16 bits k:  0319
要嵌入的水印信息是 0110001011101111111100011000010000111001001000110001010111100001011011111110111101110100010100011100100010001110011100001100100001011011101010100101001001110101100000000000111101011001110110001001011110100010001010111111110001010010010110100110001110000000011110011100011111110100010101110100011111010100110100100000100010110010110111101100110010010100100100101110000011010100001011000000001010000001
Count of 1:  190
Count of 0:  210
水印信息已保存到 semantic_mark.txt


In [32]:
import hashlib
import math

L = len(watermark)
gamma = 14000 / 400 / 4
print(gamma)
record_table = {}

cnt = 0

for index, row in data.iterrows():
    # tP_bytes = bin(row['primary_key'])[2:]
    
    k_hash = hashlib.blake2b(f"{row['primary_key']}{k}".encode()).hexdigest()
    Ks_hash = hashlib.blake2b(f"{Ks}{k_hash}".encode()).hexdigest()
    C = int(Ks_hash,16)
    if C % gamma == 0:
        zeta = int(hashlib.blake2b(f"{Ks}{k_hash}{1}".encode()).hexdigest(),16) % L
        # print(zeta)
        x = row['Cover_Type']
        new_x = math.floor((x + 5)/ 2)
        # print(C)
        # print(x,",",new_x," ")
        if watermark[zeta] == '1':
            data.at[index,'Cover_Type'] = new_x

            second_hash = hashlib.blake2b(f"{new_x}{k}{abs(x - new_x)}".encode()).hexdigest()
            # print(f"{new_x}{k}{abs(x - new_x)}")
            # print(second_hash)

            record_table[k_hash] = second_hash

        else:
            second_hash = hashlib.blake2b(f"{x}{k}".encode()).hexdigest()

            record_table[k_hash] = second_hash
       

# 将字典转换为 DataFrame
df_record_table = pd.DataFrame(list(record_table.items()), columns=['K_Hash', 'Second_Hash'])

# 保存 DataFrame 到 CSV
df_record_table.to_csv('semantic_record_table.csv', index=False)

data_file_path = '../../../../dataset/compare/semanticmark_covertype.data.csv'
data.to_csv(data_file_path, index=False)

8.75


In [33]:
import pandas as pd

file_path = '../../../../dataset/compare/semanticmark_covertype.data.csv'

# 使用列名列表读取数据
data = pd.read_csv(file_path)

# 获取Cover_Type的值域（唯一值）
unique_values = data['Cover_Type'].unique()

# 获取每个值的计数
value_counts = data['Cover_Type'].value_counts()

# 打印结果
print("Cover_Type的值域: ", unique_values)
print("\n每个值的大小:")
print(value_counts)

file_path = '../../../../dataset/compare/covtype_with_key.data'

# 使用列名列表读取数据
data = pd.read_csv(file_path)

# 获取Cover_Type的值域（唯一值）
unique_values = data['Cover_Type'].unique()

# 获取每个值的计数
value_counts = data['Cover_Type'].value_counts()

# 打印结果
print("原始Cover_Type的值域: ", unique_values)
print("\n每个值的大小:")
print(value_counts)



Cover_Type的值域:  [5 2 3 1 4 7 6]

每个值的大小:
Cover_Type
4    2688
5    2561
3    2193
6    1878
2    1607
7    1561
1    1512
Name: count, dtype: int64
原始Cover_Type的值域:  [5 2 1 7 3 6 4]

每个值的大小:
Cover_Type
5    2000
2    2000
1    2000
7    2000
3    2000
6    2000
4    2000
Name: count, dtype: int64


## 提取水印

In [34]:
import pandas as pd

# 数据集文件路径
file_path = '../../../../dataset/compare/semanticmark_covertype.data.csv'

# 使用列名列表读取数据
data = pd.read_csv(file_path)

count = [[0 for _ in range(2)] for _ in range(L)]
# print(count)
cnt = 0
for index, row in data.iterrows():
    k_hash = hashlib.blake2b(f"{row['primary_key']}{k}".encode()).hexdigest()
    Ks_hash = hashlib.blake2b(f"{Ks}{k_hash}".encode()).hexdigest()


    C = int(Ks_hash,16)
    if C % gamma == 0:
        # zeta = int(Ks_hash,16) % L
        zeta = int(hashlib.blake2b(f"{Ks}{k_hash}{1}".encode()).hexdigest(),16) % L
        x = row['Cover_Type']
        
        
        if x == 1 and record_table[k_hash] ==  hashlib.blake2b(f"{1}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 2 and record_table[k_hash] ==  hashlib.blake2b(f"{2}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 2 and record_table[k_hash] ==  hashlib.blake2b(f"{2}{k}{2}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 3 and record_table[k_hash] ==  hashlib.blake2b(f"{3}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 3 and record_table[k_hash] ==  hashlib.blake2b(f"{3}{k}{2}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 3 and record_table[k_hash] ==  hashlib.blake2b(f"{3}{k}{1}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 4 and record_table[k_hash] ==  hashlib.blake2b(f"{4}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 4 and record_table[k_hash] ==  hashlib.blake2b(f"{4}{k}{1}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 4 and record_table[k_hash] ==  hashlib.blake2b(f"{4}{k}{0}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 5 and record_table[k_hash] ==  hashlib.blake2b(f"{5}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 5 and record_table[k_hash] ==  hashlib.blake2b(f"{5}{k}{0}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 5 and record_table[k_hash] ==  hashlib.blake2b(f"{5}{k}{1}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 6 and record_table[k_hash] ==  hashlib.blake2b(f"{6}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 6 and record_table[k_hash] ==  hashlib.blake2b(f"{6}{k}{1}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 6 and record_table[k_hash] ==  hashlib.blake2b(f"{6}{k}{2}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 7 and record_table[k_hash] ==  hashlib.blake2b(f"{7}{k}".encode()).hexdigest():
            count[zeta][0] += 1
          
W_det = ""
for i in range(L):
    if count[i][0] > count[i][1]:
        W_det += '0'
    else:
        W_det += '1'

print(count)
print(watermark)
print(W_det)

def mismatch_percentage(W, W_det):
    # 首先检查 W 和 W_det 是否长度相同
    if len(W) != len(W_det):
        print('Error: The lengths of W and W_det are not the same!')
        return
    # 计算不匹配度
    count_mismatch = sum(a != b for a, b in zip(W, W_det))
    percentage = count_mismatch / len(W)
    return percentage

print(f'Mismatch Percentage: {mismatch_percentage(watermark, W_det):.2%}')


[[1, 0], [0, 0], [0, 1], [2, 0], [1, 0], [1, 0], [0, 2], [3, 0], [0, 2], [0, 0], [0, 1], [2, 0], [0, 2], [0, 1], [0, 2], [0, 2], [0, 2], [0, 2], [0, 0], [0, 0], [1, 0], [0, 0], [1, 0], [0, 2], [0, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 2], [2, 0], [0, 0], [2, 0], [1, 0], [0, 3], [0, 3], [0, 1], [2, 0], [2, 0], [0, 2], [0, 0], [0, 0], [0, 2], [2, 0], [1, 0], [1, 0], [0, 1], [0, 3], [1, 0], [1, 0], [2, 0], [0, 1], [0, 0], [0, 0], [2, 0], [0, 2], [0, 2], [0, 2], [0, 0], [0, 0], [2, 0], [0, 0], [2, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 2], [0, 3], [0, 3], [0, 1], [0, 2], [0, 0], [0, 0], [0, 1], [0, 2], [0, 2], [0, 1], [1, 0], [0, 1], [0, 2], [0, 1], [1, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [2, 0], [0, 0], [0, 2], [0, 0], [0, 2], [0, 0], [1, 0], [0, 0], [2, 0], [1, 0], [2, 0], [0, 0], [7, 0], [2, 0], [2, 0], [0, 2], [0, 1], [0, 0], [2, 0], [1, 0], [0, 2], [0, 0], [0, 1], [1, 0], [1, 0], [1, 0], [2, 0], [0, 1], [0, 2], [0, 0], [2, 0], [0, 1],